## Byte-Pair Encoding (BPE) tokenization

In [16]:
corpus = [
    "Artificial Intelligence is transforming industries.",
    "Machine Learning models require lots of data.",
    "Deep Learning uses neural networks for predictions.",
    "GPT models generate human-like text.",
    "Robotics and automation are the future."
]

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

### Frequencies of each word in the corpus

In [18]:
word_freq = {}
for text in corpus:
    words_with_offset = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offset]

    for word in new_words:
        word_freq[word] = word_freq.get(word, 0) + 1
print(word_freq)

{'Artificial': 1, 'ĠIntelligence': 1, 'Ġis': 1, 'Ġtransforming': 1, 'Ġindustries': 1, '.': 5, 'Machine': 1, 'ĠLearning': 2, 'Ġmodels': 2, 'Ġrequire': 1, 'Ġlots': 1, 'Ġof': 1, 'Ġdata': 1, 'Deep': 1, 'Ġuses': 1, 'Ġneural': 1, 'Ġnetworks': 1, 'Ġfor': 1, 'Ġpredictions': 1, 'GPT': 1, 'Ġgenerate': 1, 'Ġhuman': 1, '-': 1, 'like': 1, 'Ġtext': 1, 'Robotics': 1, 'Ġand': 1, 'Ġautomation': 1, 'Ġare': 1, 'Ġthe': 1, 'Ġfuture': 1}


### The Base Vocabulary

In [19]:
alphabet = []

for word in word_freq.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()
print(alphabet)

['-', '.', 'A', 'D', 'G', 'I', 'L', 'M', 'P', 'R', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'w', 'x', 'Ġ']


In [20]:
vocab = ["<|endoftext|>"] + alphabet.copy()
print(vocab)

['<|endoftext|>', '-', '.', 'A', 'D', 'G', 'I', 'L', 'M', 'P', 'R', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'w', 'x', 'Ġ']


### Split each word into individual characters

In [21]:
splits = {word:[c for c in word] for word in word_freq.keys()}
print(splits)

{'Artificial': ['A', 'r', 't', 'i', 'f', 'i', 'c', 'i', 'a', 'l'], 'ĠIntelligence': ['Ġ', 'I', 'n', 't', 'e', 'l', 'l', 'i', 'g', 'e', 'n', 'c', 'e'], 'Ġis': ['Ġ', 'i', 's'], 'Ġtransforming': ['Ġ', 't', 'r', 'a', 'n', 's', 'f', 'o', 'r', 'm', 'i', 'n', 'g'], 'Ġindustries': ['Ġ', 'i', 'n', 'd', 'u', 's', 't', 'r', 'i', 'e', 's'], '.': ['.'], 'Machine': ['M', 'a', 'c', 'h', 'i', 'n', 'e'], 'ĠLearning': ['Ġ', 'L', 'e', 'a', 'r', 'n', 'i', 'n', 'g'], 'Ġmodels': ['Ġ', 'm', 'o', 'd', 'e', 'l', 's'], 'Ġrequire': ['Ġ', 'r', 'e', 'q', 'u', 'i', 'r', 'e'], 'Ġlots': ['Ġ', 'l', 'o', 't', 's'], 'Ġof': ['Ġ', 'o', 'f'], 'Ġdata': ['Ġ', 'd', 'a', 't', 'a'], 'Deep': ['D', 'e', 'e', 'p'], 'Ġuses': ['Ġ', 'u', 's', 'e', 's'], 'Ġneural': ['Ġ', 'n', 'e', 'u', 'r', 'a', 'l'], 'Ġnetworks': ['Ġ', 'n', 'e', 't', 'w', 'o', 'r', 'k', 's'], 'Ġfor': ['Ġ', 'f', 'o', 'r'], 'Ġpredictions': ['Ġ', 'p', 'r', 'e', 'd', 'i', 'c', 't', 'i', 'o', 'n', 's'], 'GPT': ['G', 'P', 'T'], 'Ġgenerate': ['Ġ', 'g', 'e', 'n', 'e', 'r', '

### Frequency of each pair

In [22]:
def compute_pair_freq(splits):
    pair_freq = {}
    for word, freq in word_freq.items():
        split = splits[word]
        # print(split)
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i] , split[i+1])
            pair_freq[pair] = pair_freq.get(pair,0) + freq
    return pair_freq
pair_freq = compute_pair_freq(splits)
print(pair_freq)

{('A', 'r'): 1, ('r', 't'): 1, ('t', 'i'): 4, ('i', 'f'): 1, ('f', 'i'): 1, ('i', 'c'): 3, ('c', 'i'): 1, ('i', 'a'): 1, ('a', 'l'): 2, ('Ġ', 'I'): 1, ('I', 'n'): 1, ('n', 't'): 1, ('t', 'e'): 3, ('e', 'l'): 3, ('l', 'l'): 1, ('l', 'i'): 2, ('i', 'g'): 1, ('g', 'e'): 2, ('e', 'n'): 2, ('n', 'c'): 1, ('c', 'e'): 1, ('Ġ', 'i'): 2, ('i', 's'): 1, ('Ġ', 't'): 3, ('t', 'r'): 2, ('r', 'a'): 3, ('a', 'n'): 3, ('n', 's'): 2, ('s', 'f'): 1, ('f', 'o'): 2, ('o', 'r'): 3, ('r', 'm'): 1, ('m', 'i'): 1, ('i', 'n'): 5, ('n', 'g'): 3, ('n', 'd'): 2, ('d', 'u'): 1, ('u', 's'): 2, ('s', 't'): 1, ('r', 'i'): 1, ('i', 'e'): 1, ('e', 's'): 2, ('M', 'a'): 1, ('a', 'c'): 1, ('c', 'h'): 1, ('h', 'i'): 1, ('n', 'e'): 4, ('Ġ', 'L'): 2, ('L', 'e'): 2, ('e', 'a'): 2, ('a', 'r'): 3, ('r', 'n'): 2, ('n', 'i'): 2, ('Ġ', 'm'): 2, ('m', 'o'): 2, ('o', 'd'): 2, ('d', 'e'): 2, ('l', 's'): 2, ('Ġ', 'r'): 1, ('r', 'e'): 5, ('e', 'q'): 1, ('q', 'u'): 1, ('u', 'i'): 1, ('i', 'r'): 1, ('Ġ', 'l'): 1, ('l', 'o'): 1, ('o', 't'

### Finding the most frequent pair

In [23]:
most_freq = ""
max_freq = None

for pair, freq in pair_freq.items():
    if max_freq is None or max_freq < freq:
        most_freq = pair
        max_freq = freq
print(most_freq,max_freq)

('i', 'n') 5


In [24]:
merges = {("i","n"): "in"}
vocab.append("in")
print(vocab)

['<|endoftext|>', '-', '.', 'A', 'D', 'G', 'I', 'L', 'M', 'P', 'R', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'w', 'x', 'Ġ', 'in']


In [25]:
def merge_pair(a, b, splits):
    for word in word_freq.keys():
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i+1] == b:
                split = split[:i] + [a + b] + split[i+2:]         
            else:
                i += 1
        splits[word] = split
    return splits

In [26]:
splits = merge_pair("i","n", splits)
print(splits)

{'Artificial': ['A', 'r', 't', 'i', 'f', 'i', 'c', 'i', 'a', 'l'], 'ĠIntelligence': ['Ġ', 'I', 'n', 't', 'e', 'l', 'l', 'i', 'g', 'e', 'n', 'c', 'e'], 'Ġis': ['Ġ', 'i', 's'], 'Ġtransforming': ['Ġ', 't', 'r', 'a', 'n', 's', 'f', 'o', 'r', 'm', 'in', 'g'], 'Ġindustries': ['Ġ', 'in', 'd', 'u', 's', 't', 'r', 'i', 'e', 's'], '.': ['.'], 'Machine': ['M', 'a', 'c', 'h', 'in', 'e'], 'ĠLearning': ['Ġ', 'L', 'e', 'a', 'r', 'n', 'in', 'g'], 'Ġmodels': ['Ġ', 'm', 'o', 'd', 'e', 'l', 's'], 'Ġrequire': ['Ġ', 'r', 'e', 'q', 'u', 'i', 'r', 'e'], 'Ġlots': ['Ġ', 'l', 'o', 't', 's'], 'Ġof': ['Ġ', 'o', 'f'], 'Ġdata': ['Ġ', 'd', 'a', 't', 'a'], 'Deep': ['D', 'e', 'e', 'p'], 'Ġuses': ['Ġ', 'u', 's', 'e', 's'], 'Ġneural': ['Ġ', 'n', 'e', 'u', 'r', 'a', 'l'], 'Ġnetworks': ['Ġ', 'n', 'e', 't', 'w', 'o', 'r', 'k', 's'], 'Ġfor': ['Ġ', 'f', 'o', 'r'], 'Ġpredictions': ['Ġ', 'p', 'r', 'e', 'd', 'i', 'c', 't', 'i', 'o', 'n', 's'], 'GPT': ['G', 'P', 'T'], 'Ġgenerate': ['Ġ', 'g', 'e', 'n', 'e', 'r', 'a', 't', 'e'], '

In [27]:
vocab_size = 60

while vocab_size > len(vocab):
    pair_freq = compute_pair_freq(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freq.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [28]:
print(vocab)

['<|endoftext|>', '-', '.', 'A', 'D', 'G', 'I', 'L', 'M', 'P', 'R', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'w', 'x', 'Ġ', 'in', 're', 'ti', 'te', 'ra', 'or', 'ing', 'ne', 'Ġa', 'ic', 'li', 'ge', 'Ġt', 'ns', 'for', 'us', 'es', 'ĠL', 'ĠLe', 'ĠLea', 'ĠLear', 'ĠLearn', 'ĠLearning', 'Ġm', 'Ġmo']


In [29]:
print(merges)

{('i', 'n'): 'in', ('r', 'e'): 're', ('t', 'i'): 'ti', ('t', 'e'): 'te', ('r', 'a'): 'ra', ('o', 'r'): 'or', ('in', 'g'): 'ing', ('n', 'e'): 'ne', ('Ġ', 'a'): 'Ġa', ('i', 'c'): 'ic', ('l', 'i'): 'li', ('g', 'e'): 'ge', ('Ġ', 't'): 'Ġt', ('n', 's'): 'ns', ('f', 'or'): 'for', ('u', 's'): 'us', ('e', 's'): 'es', ('Ġ', 'L'): 'ĠL', ('ĠL', 'e'): 'ĠLe', ('ĠLe', 'a'): 'ĠLea', ('ĠLea', 'r'): 'ĠLear', ('ĠLear', 'n'): 'ĠLearn', ('ĠLearn', 'ing'): 'ĠLearning', ('Ġ', 'm'): 'Ġm', ('Ġm', 'o'): 'Ġmo'}


### Apply all

In [30]:
def tokenize(text):
    word_with_offset = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in word_with_offset]
    splits = [[l for l in word] for word in new_words]
    
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) -1:
                if split[i] == pair[0] and split[i+1] == pair[1]:
                    split = split[:i] + [merge] + split[i+2:]
                else:
                    i += 1
            splits[idx] = split
    return sum(splits, [])
print(tokenize("I like Learning ML and Robotics"))
    

['I', 'Ġ', 'li', 'k', 'e', 'ĠLearning', 'Ġ', 'M', 'L', 'Ġa', 'n', 'd', 'Ġ', 'R', 'o', 'b', 'o', 'ti', 'c', 's']
